In [1]:
import pandas as pd
from itertools import product

In [2]:
df_original = pd.read_csv("sales_train_dates_added.csv")
df_original.columns

Index(['Unnamed: 0', 'date_x', 'date_block_num', 'shop_id', 'item_id',
       'item_price', 'item_cnt_day', 'date_y', 'day_name', 'day_of_week',
       'day_of_month', 'week', 'month_name', 'month_of_year',
       'month_year_name', 'month_year_name_sort', 'quarter',
       'quarter_year_name_sort', 'year', 'year_half', 'red_day_not_sun',
       'black_friday', 'Date_formatted'],
      dtype='object')

In [3]:
#BlackFriday, Holiday, Red Day
#month_of_year, quarter, red_day_not_sun, black_friday, , year_half

df_draft = df_original[["date_block_num", "shop_id", "item_id", "item_cnt_day","month_of_year", "quarter", "red_day_not_sun", "black_friday", "year_half"]].copy()
df_draft = df_draft.groupby(["date_block_num", "item_id", "shop_id"], as_index=False).sum()
df_result = pd.DataFrame()
for i in range(34):
    df_date_block = df_draft[df_draft["date_block_num"] == i][["date_block_num", "item_id", "shop_id"]]
    df_crossed = pd.DataFrame(list(product([i], df_date_block["item_id"].unique(), df_date_block["shop_id"].unique())),
                               columns=["date_block_num", "item_id", "shop_id"])
    df_result = pd.concat([df_result, df_crossed], ignore_index=True)
df_result = pd.merge(df_result, df_draft, on=["date_block_num", "item_id", "shop_id"], how="left")
df_result["item_cnt_day"].fillna(0, inplace=True)
df_result

,date_block_num,item_id,shop_id,item_cnt_day
0,0,19,25,1.0
1,0,19,1,0.0
2,0,19,2,0.0
3,0,19,10,0.0
4,0,19,19,0.0
...,...,...,...,...
10913845,33,22167,34,0.0
10913846,33,22167,41,0.0
10913847,33,22167,48,0.0
10913848,33,22167,9,0.0


In [4]:
df_sum = df_result.groupby(["date_block_num", "item_id", "shop_id"], as_index=False)["item_cnt_day"].sum()

In [5]:
df_lag = df_sum.copy()
df_lag["lag_1"] = df_lag["date_block_num"] + 1
df_lag.isna().any()

date_block_num    False
item_id           False
shop_id           False
item_cnt_day      False
lag_1             False
dtype: bool

In [6]:
df_lag = pd.merge(df_sum, df_lag, left_on=["date_block_num", "item_id", "shop_id"], right_on=["lag_1", "item_id", "shop_id"], how="left")
df_lag = df_lag.sort_values("item_cnt_day_x", ascending=False)
df_lag = df_lag.drop(columns=["date_block_num_y", "lag_1"])
df_lag["item_cnt_day_y"].fillna(0, inplace=True)

In [7]:
df_lag = df_lag.rename(columns={"item_cnt_day_x": "item_cnt_month", "item_cnt_day_y": "item_cnt_last_month"})
df_lag

,date_block_num_x,item_id,shop_id,item_cnt_month,item_cnt_last_month
10794398,33,11373,12,2253.0,246.0
10545641,32,9249,55,1644.0,19.0
10545567,32,9248,12,1644.0,19.0
4471622,11,20949,31,1305.0,989.0
9341405,27,3730,55,1242.0,0.0
...,...,...,...,...,...
724919,1,21323,28,-4.0,4.0
6986169,19,11367,12,-5.0,0.0
18549,0,1593,12,-6.0,0.0
18414,0,1590,12,-13.0,0.0


In [8]:
df_lag_12 = df_sum.copy()
df_lag_12["lag_12"] = df_lag_12["date_block_num"]+12
df_lag_12

,date_block_num,item_id,shop_id,item_cnt_day,lag_12
0,0,19,0,0.0,12
1,0,19,1,0.0,12
2,0,19,2,0.0,12
3,0,19,3,0.0,12
4,0,19,4,0.0,12
...,...,...,...,...,...
10913845,33,22167,55,0.0,45
10913846,33,22167,56,0.0,45
10913847,33,22167,57,0.0,45
10913848,33,22167,58,0.0,45


In [9]:
df_lag = pd.merge(df_lag, df_lag_12, left_on=["date_block_num_x", "item_id", "shop_id"], right_on=["lag_12", "item_id", "shop_id"], how="left")

In [10]:
df_lag = df_lag.drop(columns=["date_block_num", "lag_12"])
df_lag = df_lag.rename(columns={"item_cnt_day": "item_cnt_last_year"})
df_lag


,date_block_num_x,item_id,shop_id,item_cnt_month,item_cnt_last_month,item_cnt_last_year
0,33,11373,12,2253.0,246.0,68.0
1,32,9249,55,1644.0,19.0,NaN
2,32,9248,12,1644.0,19.0,NaN
3,11,20949,31,1305.0,989.0,NaN
4,27,3730,55,1242.0,0.0,NaN
...,...,...,...,...,...,...
10913845,1,21323,28,-4.0,4.0,NaN
10913846,19,11367,12,-5.0,0.0,NaN
10913847,0,1593,12,-6.0,0.0,NaN
10913848,0,1590,12,-13.0,0.0,NaN


In [11]:
df_lag["item_cnt_last_year"].fillna(0, inplace=True)
df_lag

,date_block_num_x,item_id,shop_id,item_cnt_month,item_cnt_last_month,item_cnt_last_year
0,33,11373,12,2253.0,246.0,68.0
1,32,9249,55,1644.0,19.0,0.0
2,32,9248,12,1644.0,19.0,0.0
3,11,20949,31,1305.0,989.0,0.0
4,27,3730,55,1242.0,0.0,0.0
...,...,...,...,...,...,...
10913845,1,21323,28,-4.0,4.0,0.0
10913846,19,11367,12,-5.0,0.0,0.0
10913847,0,1593,12,-6.0,0.0,0.0
10913848,0,1590,12,-13.0,0.0,0.0
